In [24]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

In [25]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5990976684985645884
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16514700651497956748
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-19 21:37:45.764094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:37:45.764170: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:37:45.764189: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:37:45.764514: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:37:45.764530: I tensorflow/core/common_runtime/gpu/gpu

In [26]:
# Load the training and testing data:
train_values = np.empty(shape=[0, length])
test_values = np.empty(shape=[0, length])

train_beats = glob.glob('./data/train_test/train_patients.csv')
test_beats = glob.glob('./data/train_test/test_patients.csv')

for j in train_beats:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    train_values = np.append(train_values, csvrows, axis=0)

for j in test_beats:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    test_values = np.append(test_values, csvrows, axis=0)
    
print(train_values.shape)
print(test_values.shape)

# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42,stratify=y_train)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


Loading  ./data/train_test/train_patients.csv
Loading  ./data/train_test/test_patients.csv
(38752, 277)
(72795, 277)
X_train shape: (27126, 275)
X_val shape: (11626, 275)
y_train shape: (27126,)
y_val shape: (11626,)


In [27]:
from keras.layers import LSTM, Dropout, Dense,Flatten
from keras.optimizers import Nadam
from keras.losses import categorical_crossentropy  
from keras.utils import to_categorical
from keras import backend as K

def focal_loss(gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        loss = -K.pow(1 - pt, gamma) * K.log(pt)
        return loss

    return focal_loss_fixed


# Initialize the model
model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=(X_train1.shape[1],X_train1.shape[2])))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
# Add a Dense layer
model.add(Dense(7, activation='softmax'))  # Assuming 10 classes for classification

# Compile the model with focal loss
model.compile(loss=focal_loss(gamma=2.0), optimizer=Nadam(), metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 275, 64)           16896     
                                                                 
 lstm_13 (LSTM)              (None, 275, 64)           33024     
                                                                 
 lstm_14 (LSTM)              (None, 275, 64)           33024     
                                                                 
 lstm_15 (LSTM)              (None, 275, 64)           33024     
                                                                 
 flatten_3 (Flatten)         (None, 17600)             0         
                                                                 
 dense_3 (Dense)             (None, 7)                 123207    
                                                                 
Total params: 239175 (934.28 KB)
Trainable params: 239

In [28]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


In [29]:
class_weights

array([0.25926138, 2.33402168, 2.33281734, 2.33281734, 2.33402168,
       2.33281734])

In [30]:

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min',min_delta=0.001, restore_best_weights=True)
history = model.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)




Epoch 1/50
848/848 [==============================] - 64s 72ms/step - loss: 0.0269 - accuracy: 0.9045 - val_loss: 0.0161 - val_accuracy: 0.9453
Epoch 2/50
848/848 [==============================] - 61s 72ms/step - loss: 0.0135 - accuracy: 0.9513 - val_loss: 0.0104 - val_accuracy: 0.9626
Epoch 3/50
848/848 [==============================] - 63s 74ms/step - loss: 0.0102 - accuracy: 0.9617 - val_loss: 0.0085 - val_accuracy: 0.9692
Epoch 4/50
848/848 [==============================] - 67s 79ms/step - loss: 0.0087 - accuracy: 0.9674 - val_loss: 0.0079 - val_accuracy: 0.9702
Epoch 5/50
848/848 [==============================] - 72s 85ms/step - loss: 0.0074 - accuracy: 0.9713 - val_loss: 0.0066 - val_accuracy: 0.9766
Epoch 6/50
848/848 [==============================] - 73s 86ms/step - loss: 0.0063 - accuracy: 0.9754 - val_loss: 0.0094 - val_accuracy: 0.9658
Epoch 7/50
848/848 [==============================] - 70s 83ms/step - loss: 0.0056 - accuracy: 0.9772 - val_loss: 0.0059 - val_accuracy:

In [31]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[    0 58952  2493  6725  2923  1277   425]


In [32]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = model.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




2275/2275 [==============================] - 69s 30ms/step
Confusion Matrix:
[[58351     2    14   198   312    75]
 [    1  2488     0     4     0     0]
 [   40     0  6626     6    46     7]
 [   49     0     4  2833    10    27]
 [  157     0     3     8  1109     0]
 [   15     0     0     7     1   402]]
F1 Score: 0.9868334134929451


In [33]:
model.save('./models/lstmmodel.h5')
